In [1]:
# Clonar repositorio
!git clone https://github.com/mdegano-fiuba/MIA_VPC3.git

Cloning into 'MIA_VPC3'...
remote: Enumerating objects: 1113, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 1113 (delta 38), reused 171 (delta 1), pack-reused 829 (from 1)
Receiving objects: 100% (1113/1113), 208.48 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (321/321), done.


In [3]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración: ruta a la carpeta de métricas
RUN_PATH = "/content/MIA_VPC3/project/mlflow/mlruns/292277474786075902/94b0390cd1ee449196a63dd52fbe5a19/metrics/"

METRICS_FILES = {
    "EVAL_ACCURACY": RUN_PATH + "eval_accuracy",
    "EVAL_F1": RUN_PATH + "eval_f1",
    "EVAL_PRECISION": RUN_PATH + "eval_precision",
    "EVAL_RECALL": RUN_PATH + "eval_recall"
}

In [4]:
# Leer cada archivo y almacenar en diccionario
metrics_data = {}
for metric_name, file_path in METRICS_FILES.items():
    data = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                try:
                    value = float(parts[1])
                    step = int(parts[2])
                    data.append((step, value))
                except ValueError:
                    continue
    df = pd.DataFrame(data, columns=["step", "value"])
    metrics_data[metric_name] = df

In [5]:
# Crear subplots: un subplot por métrica
n_metrics = len(metrics_data)
fig = make_subplots(rows=n_metrics, cols=1, shared_xaxes=True,
                    subplot_titles=list(metrics_data.keys()))

# Graficar cada métrica
for i, (metric_name, df) in enumerate(metrics_data.items(), start=1):
    fig.add_trace(
        go.Scatter(
            x=df["step"],
            y=df["value"],
            mode="lines+markers",
            name=metric_name,
            hovertemplate=f"{metric_name}: %{{y:.3f}}<br>Step: %{{x}}"
        ),
        row=i, col=1
    )

    # Líneas de máximo y mínimo
    fig.add_hline(y=df["value"].max(), line=dict(color="orange", dash="dash"),
                  row=i, col=1, annotation_text=f"Max: {df['value'].max():.3f}", annotation_position="top left")
    fig.add_hline(y=df["value"].min(), line=dict(color="red", dash="dash"),
                  row=i, col=1, annotation_text=f"Min: {df['value'].min():.3f}", annotation_position="bottom left")

    # Último valor
    fig.add_trace(
        go.Scatter(
            x=[df["step"].iloc[-1]],
            y=[df["value"].iloc[-1]],
            mode="markers+text",
            text=[f"Último: {df['value'].iloc[-1]:.3f}"],
            textposition="top right",
            marker=dict(color="green", size=10),
            showlegend=False
        ),
        row=i, col=1
    )

# Ajustes de layout
fig.update_layout(
    height=300*n_metrics,
    width=900,
    title_text="Dashboard interactivo de métricas de evaluación",
    hovermode="x unified"
)

fig.show()


In [6]:
# Crear gráfico de zoom
fig = go.Figure()

for metric_name, df in metrics_data.items():
    fig.add_trace(
        go.Scatter(
            x=df["step"],
            y=df["value"],
            mode="lines+markers",
            name=metric_name,
            hovertemplate=f"{metric_name}: %{{y:.3f}}<br>Step: %{{x}}"
        )
    )

# Ajustar rango Y para ver micro-variaciones
fig.update_yaxes(range=[0.975, 0.999])

# Layout
fig.update_layout(
    title="Zoom en métricas (rango 0.975–0.995) para analizar micro-fluctuaciones",
    xaxis_title="Step",
    yaxis_title="Valor de métrica",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
